<a href="https://colab.research.google.com/github/nwon24/nlp/blob/main/W4/amazon_review_polarity_csv/cnn_Amazon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN Using PyTorch

For some reason, the cost never goes below around 0.69. :(

In [8]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

device=torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

data=pd.read_csv("xab",encoding="utf-8")

tfidf  = TfidfVectorizer()
corpus = tfidf.fit_transform(data["text"])
train_corpus, test_corpus, train_label, test_label = model_selection.train_test_split(data["text"],data['label'],test_size=0.3)
Encoder = LabelEncoder()
train_label = Encoder.fit_transform(train_label)
test_label = Encoder.fit_transform(test_label)
train_corpus_tfidf=tfidf.transform(train_corpus)
test_corpus_tfidf=tfidf.transform(test_corpus)

bsize=50
epochs=10
lr=1e-5

class dense_classifier(nn.Module):
    def __init__(self,n):
        super(dense_classifier,self).__init__()
        self.n=n
        self.network=nn.Sequential(
            nn.Linear(self.n,360),
            nn.Linear(360,1),
            nn.Sigmoid()
        )
    def forward(self,x):
        return self.network(x)

class classifier(nn.Module):
    def __init__(self,n,kerns,pool_kerns,out_chans):
        super(classifier,self).__init__()
        self.kerns=kerns
        self.pool_kerns=pool_kerns
        self.convs=[]
        self.pools=[]
        self.stride=1
        self.n=n
        #self.embed=nn.Embedding(n,max_length)
        in_chan=1
        for i in range(len(kerns)):
            out_chan=out_chans[i]
            kern=kerns[i]
            pool_kern=pool_kerns[i]
            self.convs.append(nn.Conv1d(in_chan,out_chan,kern))
            self.compute_size(kern,self.stride)
            self.pools.append(nn.MaxPool1d(pool_kern))
            self.compute_size(pool_kern,pool_kern)
            in_chan=out_chan
        self.convs=nn.ModuleList(self.convs)
        self.pools=nn.ModuleList(self.pools)
        self.fc1=nn.Linear(self.n*out_chan,100)
        self.fc2=nn.Linear(100,1)

    def compute_size(self,kernel,stride):
        self.n=(self.n-kernel)//stride+1
        return self.n

    def forward(self,x):
        #x=self.embed(x)
        for i in range(len(self.kerns)):
            x=nn.ReLU()(self.convs[i](x))
            x=self.pools[i](x)
        x=torch.flatten(x,1)
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        #x=nn.Sigmoid()(x)
        return x

class corpus(Dataset):
    def __init__(self,corpus,label):
        self.corpus=corpus
        self.label=label
    def __len__(self):
        return len(self.corpus)
    def __getitem__(self,idx):
        return self.corpus[idx],self.label[idx]

def train(dataloader,model,loss_fn,optimizer):
    size=len(dataloader.dataset)
    model.train()
    for batch,(x,y) in enumerate(dataloader):
        x,y=x.to(device),y.to(device)
        #print(torch.sum(x[0]!=0))
        #pred=model(x.reshape(bsize,1,x.shape[1]).to(device))
        pred=model(x)
        cost=loss_fn(pred.flatten(),y)
        cost.backward()
        optimizer.step()
        optimizer.zero_grad()
        if batch % 10 == 0:
            cost, current = cost.item(), batch * bsize + len(x)
            print(f"cost: {cost:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for x, y in dataloader:
            x,y=x.to(device),y.to(device)
            #pred=model(x.reshape(bsize,1,x.shape[1]).to(device))
            pred=model(x)
            print(pred,y)
            test_loss += loss_fn(pred.flatten(), y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

text="ajbjdfowd"
#input =torch.from_numpy(tfidf.transform([text]).toarray()).float()
train_corpus_data=torch.from_numpy(train_corpus_tfidf.toarray()).float()
test_corpus_data=torch.from_numpy(test_corpus_tfidf.toarray()).float()
train_label_data=torch.from_numpy(train_label).float()
test_label_data=torch.from_numpy(test_label).float()

train_loader=DataLoader(corpus(train_corpus_data,train_label_data),batch_size=bsize)
test_loader=DataLoader(corpus(test_corpus_data,test_label_data),batch_size=bsize)

#input=input.reshape(1,input.shape[0],input.shape[1]
#c=classifier(train_corpus_data.shape[1],[2,3,4,5],[2,2,2,2],[6,16,24,32])
cnn_classifier=classifier(train_corpus_data.shape[1],[2],[2],[3]).to(device)
#input=input.reshape(1,input.shape[0],input.shape[1])
#print(c.forward(input))
dense_classifier=dense_classifier(train_corpus_data.shape[1]).to(device)

c=dense_classifier

loss=nn.BCELoss()
#loss=nn.BCEWithLogitsLoss()
#optim=torch.optim.SGD(c.parameters(),momentum=0.8,lr=lr)
optim=torch.optim.Adam(c.parameters(),lr=lr)
#optim=torch.optim.ASGD(c.parameters(),lr=lr)

#print(torch.sum(train_label_data==1))

for epoch in range(epochs):
    train(train_loader,c,loss,optim)
test_loop(test_loader,c,loss)

Streaming output truncated to the last 5000 lines.
        [0.4888],
        [0.4374],
        [0.4578],
        [0.5206],
        [0.5577],
        [0.4966],
        [0.5399],
        [0.5151],
        [0.5095],
        [0.5117],
        [0.4932],
        [0.5179]], device='cuda:0') tensor([1., 1., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0.,
        1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1.,
        1., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1.],
       device='cuda:0')
tensor([[0.5330],
        [0.4708],
        [0.4568],
        [0.5182],
        [0.5399],
        [0.5006],
        [0.4820],
        [0.5209],
        [0.4513],
        [0.5115],
        [0.5154],
        [0.5478],
        [0.3880],
        [0.4836],
        [0.5709],
        [0.3722],
        [0.5439],
        [0.5568],
        [0.5512],
        [0.5134],
        [0.5490],
        [0.4610],
        [0.5033],
        [0.5062],
        [0.5293],
        [0.5872]

# Keras

Yay, it works! (A bit of overfitting is happening, but at least the first few epochs look good.)

In [1]:
import os
os.environ["KERAS_BACKEND"] = "torch"
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers import InputLayer
from keras.preprocessing import sequence
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection


data=pd.read_csv("xab",encoding="utf-8")

tfidf  = TfidfVectorizer()
corpus = tfidf.fit_transform(data["text"])
train_corpus, test_corpus, train_label, test_label = model_selection.train_test_split(data["text"],data["label"],test_size=0.4)
Encoder = LabelEncoder()
train_label = Encoder.fit_transform(train_label)
test_label = Encoder.fit_transform(test_label)
train_corpus_tfidf=tfidf.transform(train_corpus)
test_corpus_tfidf=tfidf.transform(test_corpus)

bsize=32
epochs=8
lr=1e-5

features=train_corpus_tfidf.shape[1]
train_corpus_tfidf=train_corpus_tfidf.toarray()
test_corpus_tfidf=test_corpus_tfidf.toarray()
train_corpus_tfidf=train_corpus_tfidf.reshape(train_corpus_tfidf.shape[0],train_corpus_tfidf.shape[1],1)
test_corpus_tfidf=test_corpus_tfidf.reshape(test_corpus_tfidf.shape[0],test_corpus_tfidf.shape[1],1)

model=Sequential([InputLayer(shape=(features,1),batch_size=bsize),
                  Conv1D(3,2,activation="relu"),
                  MaxPooling1D(),
                  Dropout(0.2),
                  #Conv1D(32,16,activation="relu"),
                  #MaxPooling1D(),
                  #Dropout(0.2),
                  #Conv1D(3,2,activation="relu"),
                  #MaxPooling1D(),
                  #Dropout(0.2),
                  #Conv1D(36,4,activation="relu"),
                  #MaxPooling1D(),
                  #Dropout(0.2),
                  #Conv1D(18,8,activation="relu"),
                  #MaxPooling1D(),
                  #Dropout(0.2),
                  Flatten(),
                  #Dense(100,activation="relu"),
                  #Dense(50,activation="relu"),
                  Dense(1,activation="sigmoid")
                  ])
#model=Sequential([InputLayer(shape=(features,)),
#                  Dense(100,activation="relu"),
#                  Dense(1,activation="sigmoid")
#                  ])
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
model.summary()

model.fit(train_corpus_tfidf, train_label, epochs=epochs, batch_size=bsize, validation_data=(test_corpus_tfidf, test_label),shuffle=True)
eval_loss,eval_accuracy=model.evaluate(test_corpus_tfidf,test_label)
print(eval_loss,eval_accuracy)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (32, 47200, 3)         │             9 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (32, 23600, 3)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (32, 23600, 3)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (32, 70800)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (32, 1)                │        70,801 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 70,810 (276.60 KB)

 Trainable params: 70,810 (276.60 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/8
375/375 ━━━━━━━━━━━━━━━━━━━━ 14s 28ms/step - accuracy: 0.6669 - loss: 0.6799 - val_accuracy: 0.8040 - val_loss: 0.6203
Epoch 2/8
375/375 ━━━━━━━━━━━━━━━━━━━━ 11s 26ms/step - accuracy: 0.8527 - loss: 0.5747 - val_accuracy: 0.8296 - val_loss: 0.5223
Epoch 3/8
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 26ms/step - accuracy: 0.8722 - loss: 0.4585 - val_accuracy: 0.8371 - val_loss: 0.4393
Epoch 4/8
375/375 ━━━━━━━━━━━━━━━━━━━━ 11s 26ms/step - accuracy: 0.8980 - loss: 0.3628 - val_accuracy: 0.8444 - val_loss: 0.3925
Epoch 5/8
375/375 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - accuracy: 0.9161 - loss: 0.2968 - val_accuracy: 0.8486 - val_loss: 0.3688
Epoch 6/8
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.9275 - loss: 0.2501 - val_accuracy: 0.8491 - val_loss: 0.3546
Epoch 7/8
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 26ms/step - accuracy: 0.9356 - loss: 0.2208 - val_accuracy: 0.8490 - val_loss: 0.3491
Epoch 8/8
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 26ms/step - accuracy: 0.9503 - loss: 0.1885 - val_accu